In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import gym

In [2]:
# Hyper Parameters
BATCH_SIZE = 32
LR = 0.01
EPSILON = 0.9 # greedy policy
GAMMA = 0.9 # alpha
TARGET_REPLACE_ITER = 100 # target update frequency
MEMORY_CAPACITY = 2000
env = gym.make('CartPole-v0')
N_ACTIONS = env.action_space.n
N_STATES = env.observation_space.shape[0]
ENV_A_SHAPE = 0 if isinstance(env.action_space.sample(), int) else env.action_space.sample().shape     # to confirm the shape
# using GPU
print("GPU is ->",torch.cuda.is_available())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(torch.cuda.get_device_name(0))


WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
GPU is -> True
GeForce GTX 1080


/home/rozen/gym/gym/__init__.py:22: UserWarning: DEPRECATION WARNING: to improve load times, gym no longer automatically loads gym.spaces. Please run "import gym.spaces" to load gym.spaces on your own. This warning will turn into an error in a future version of gym.
  warnings.warn('DEPRECATION WARNING: to improve load times, gym no longer automatically loads gym.spaces. Please run "import gym.spaces" to load gym.spaces on your own. This warning will turn into an error in a future version of gym.')


In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(N_STATES, 50)
        self.fc1.weight.data.normal_(0,0.1)
        self.out = nn.Linear(50, N_ACTIONS)
        self.out.weight.data.normal_(0,0.1)
    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        actions_value = self.out(x)
        return actions_value


In [4]:
class DQN:
    def __init__(self):
        self.eval_net, self.target_net = Net().to(device), Net().to(device)
        #self.eval_net.cuda()
        #self.target_net.cuda()
        
        self.learn_step_counter = 0 # for target update
        self.memory_counter = 0 # for storing memory
        self.memory = np.zeros((MEMORY_CAPACITY, N_STATES*2+2))
        self.optimizer = torch.optim.Adam(self.eval_net.parameters(), lr=LR)
        self.loss_func = nn.MSELoss()
    def choose_action(self,x):
        x = torch.unsqueeze(torch.FloatTensor(x),0).to(device) # input data to tensor
        if np.random.uniform() < EPSILON:
            action_value = self.eval_net.forward(x)
            action = torch.max(action_value, 1)[1].data.cpu().numpy()
            action = action[0] if ENV_A_SHAPE == 0 else action.reshape(ENV_A_SHAPE) 
        else:
            action = np.random.randint(0, N_ACTIONS)
            action = action if ENV_A_SHAPE == 0 else action.reshape(ENV_A_SHAPE)
        return action
    def store_transition(self,s,a,r,s_):
        transition = np.hstack((s, [a,r],s_))
        index = self.memory_counter % MEMORY_CAPACITY
        self.memory[index,:] = transition
        self.memory_counter += 1
    def learn(self):
        # target parameter update
        if self.learn_step_counter % TARGET_REPLACE_ITER == 0:
            self.target_net.load_state_dict(self.eval_net.state_dict())
        self.learn_step_counter +=1
        
        # sample batch transitions
        sample_index = np.random.choice(MEMORY_CAPACITY, BATCH_SIZE)
        b_memory = self.memory[sample_index, :]
        # need using cuda
        b_s = torch.FloatTensor(b_memory[:, :N_STATES]).cuda()
        b_a = torch.LongTensor(b_memory[:, N_STATES:N_STATES+1].astype(int)).cuda()
        b_r = torch.FloatTensor(b_memory[:, N_STATES+1:N_STATES+2]).cuda()
        b_s_ = torch.FloatTensor(b_memory[:, -N_STATES:]).cuda()
         # q_eval w.r.t the action in experience
        q_eval = self.eval_net(b_s).gather(1, b_a)  # shape (batch, 1)
        q_next = self.target_net(b_s_).detach()     # detach from graph, don't backpropagate
        q_target = b_r + GAMMA * q_next.max(1)[0].view(BATCH_SIZE, 1)   # shape (batch, 1)
        loss = self.loss_func(q_eval, q_target)

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

In [5]:
dqn = DQN()


print('\nCollecting experience...')
for i_episode in range(1000):
    s = env.reset()
    ep_r = 0
    while True:
        #env.render()
        a = dqn.choose_action(s)

        # take action
        s_, r, done, info = env.step(a)

        # modify the reward
        x, x_dot, theta, theta_dot = s_
        #r1 = (env.unwrapped.x_threshold - abs(x)) / env.unwrapped.x_threshold - 0.8
        #r2 = (env.unwrapped.theta_threshold_radians - abs(theta)) / env.unwrapped.theta_threshold_radians - 0.5
        #r = r1 + r2

        dqn.store_transition(s, a, r, s_)

        ep_r += r
        if dqn.memory_counter > MEMORY_CAPACITY:
            dqn.learn()
            if done:
                print('Ep: ', i_episode,
                      '| Ep_r: ', round(ep_r, 2))

        if done:
            break
        s = s_


Ep:  197 | Ep_r:  11.0
Ep:  198 | Ep_r:  10.0
Ep:  199 | Ep_r:  14.0
Ep:  200 | Ep_r:  18.0
Ep:  201 | Ep_r:  11.0
Ep:  202 | Ep_r:  9.0
Ep:  203 | Ep_r:  10.0
Ep:  204 | Ep_r:  12.0
Ep:  205 | Ep_r:  11.0
Ep:  206 | Ep_r:  9.0
Ep:  207 | Ep_r:  22.0
Ep:  208 | Ep_r:  11.0
Ep:  209 | Ep_r:  32.0
Ep:  210 | Ep_r:  18.0
Ep:  211 | Ep_r:  11.0
Ep:  212 | Ep_r:  10.0
Ep:  213 | Ep_r:  9.0
Ep:  214 | Ep_r:  13.0
Ep:  215 | Ep_r:  25.0
Ep:  216 | Ep_r:  18.0
Ep:  217 | Ep_r:  9.0
Ep:  218 | Ep_r:  10.0
Ep:  219 | Ep_r:  10.0
Ep:  220 | Ep_r:  11.0
Ep:  221 | Ep_r:  9.0
Ep:  222 | Ep_r:  26.0
Ep:  223 | Ep_r:  11.0
Ep:  224 | Ep_r:  8.0
Ep:  225 | Ep_r:  9.0
Ep:  226 | Ep_r:  9.0
Ep:  227 | Ep_r:  10.0
Ep:  228 | Ep_r:  11.0
Ep:  229 | Ep_r:  22.0
Ep:  230 | Ep_r:  11.0
Ep:  231 | Ep_r:  11.0
Ep:  232 | Ep_r:  11.0
Ep:  233 | Ep_r:  9.0
Ep:  234 | Ep_r:  13.0
Ep:  235 | Ep_r:  10.0
Ep:  236 | Ep_r:  9.0
Ep:  237 | Ep_r:  10.0
Ep:  238 | Ep_r:  9.0
Ep:  239 | Ep_r:  9.0
Ep:  240 | Ep_r:  21.0

Ep:  560 | Ep_r:  12.0
Ep:  561 | Ep_r:  10.0
Ep:  562 | Ep_r:  10.0
Ep:  563 | Ep_r:  9.0
Ep:  564 | Ep_r:  10.0
Ep:  565 | Ep_r:  9.0
Ep:  566 | Ep_r:  8.0
Ep:  567 | Ep_r:  8.0
Ep:  568 | Ep_r:  11.0
Ep:  569 | Ep_r:  9.0
Ep:  570 | Ep_r:  12.0
Ep:  571 | Ep_r:  10.0
Ep:  572 | Ep_r:  9.0
Ep:  573 | Ep_r:  9.0
Ep:  574 | Ep_r:  10.0
Ep:  575 | Ep_r:  9.0
Ep:  576 | Ep_r:  10.0
Ep:  577 | Ep_r:  10.0
Ep:  578 | Ep_r:  9.0
Ep:  579 | Ep_r:  10.0
Ep:  580 | Ep_r:  9.0
Ep:  581 | Ep_r:  10.0
Ep:  582 | Ep_r:  10.0
Ep:  583 | Ep_r:  10.0
Ep:  584 | Ep_r:  11.0
Ep:  585 | Ep_r:  9.0
Ep:  586 | Ep_r:  9.0
Ep:  587 | Ep_r:  9.0
Ep:  588 | Ep_r:  10.0
Ep:  589 | Ep_r:  10.0
Ep:  590 | Ep_r:  11.0
Ep:  591 | Ep_r:  10.0
Ep:  592 | Ep_r:  10.0
Ep:  593 | Ep_r:  9.0
Ep:  594 | Ep_r:  9.0
Ep:  595 | Ep_r:  9.0
Ep:  596 | Ep_r:  10.0
Ep:  597 | Ep_r:  9.0
Ep:  598 | Ep_r:  10.0
Ep:  599 | Ep_r:  10.0
Ep:  600 | Ep_r:  10.0
Ep:  601 | Ep_r:  10.0
Ep:  602 | Ep_r:  11.0
Ep:  603 | Ep_r:  10.0
Ep:  

Ep:  924 | Ep_r:  9.0
Ep:  925 | Ep_r:  11.0
Ep:  926 | Ep_r:  10.0
Ep:  927 | Ep_r:  15.0
Ep:  928 | Ep_r:  9.0
Ep:  929 | Ep_r:  11.0
Ep:  930 | Ep_r:  12.0
Ep:  931 | Ep_r:  11.0
Ep:  932 | Ep_r:  8.0
Ep:  933 | Ep_r:  11.0
Ep:  934 | Ep_r:  10.0
Ep:  935 | Ep_r:  10.0
Ep:  936 | Ep_r:  10.0
Ep:  937 | Ep_r:  9.0
Ep:  938 | Ep_r:  8.0
Ep:  939 | Ep_r:  10.0
Ep:  940 | Ep_r:  8.0
Ep:  941 | Ep_r:  9.0
Ep:  942 | Ep_r:  12.0
Ep:  943 | Ep_r:  9.0
Ep:  944 | Ep_r:  10.0
Ep:  945 | Ep_r:  9.0
Ep:  946 | Ep_r:  10.0
Ep:  947 | Ep_r:  8.0
Ep:  948 | Ep_r:  9.0
Ep:  949 | Ep_r:  9.0
Ep:  950 | Ep_r:  9.0
Ep:  951 | Ep_r:  8.0
Ep:  952 | Ep_r:  11.0
Ep:  953 | Ep_r:  9.0
Ep:  954 | Ep_r:  10.0
Ep:  955 | Ep_r:  9.0
Ep:  956 | Ep_r:  10.0
Ep:  957 | Ep_r:  10.0
Ep:  958 | Ep_r:  9.0
Ep:  959 | Ep_r:  9.0
Ep:  960 | Ep_r:  9.0
Ep:  961 | Ep_r:  10.0
Ep:  962 | Ep_r:  10.0
Ep:  963 | Ep_r:  11.0
Ep:  964 | Ep_r:  9.0
Ep:  965 | Ep_r:  9.0
Ep:  966 | Ep_r:  9.0
Ep:  967 | Ep_r:  8.0
Ep:  968 | 